# Composing Contracts

Toying with ideas from [Composing Contracts:An Adventure in Financial Engineering](https://www.cs.tufts.edu/~nr/cs257/archive/simon-peyton-jones/contracts.pdf) by Jones, Eber and Seward.

In [34]:
from typing import TypeVar, Generic
from dataclasses import dataclass, asdict, astuple
from abc import ABC, abstractmethod
from datetime import datetime

In [2]:
class Observable(ABC):
    pass

In [3]:
@dataclass
class Konst(Observable):
    constant: float

In [4]:
class Contract(ABC):
    pass

In [9]:
@dataclass
class Zero(Contract):
    pass

@dataclass
class One(Contract):
    currency: str

@dataclass
class Give(Contract):
    contract: Contract

@dataclass
class And(Contract):
    contract1: Contract
    contract2: Contract

@dataclass
class Or(Contract):
    contract1: Contract
    contract2: Contract

@dataclass
class Truncate(Contract):
    horizon: datetime
    contract: Contract

@dataclass
class Then(Contract):
    contract1: Contract
    contract2: Contract

@dataclass
class Scale(Contract):
    observable: Observable
    contract: Contract

@dataclass
class Get(Contract):
    contract: Contract

@dataclass
class Anytime(Contract):
    contract: Contract

In [36]:
T = TypeVar('T')

class ContractVisitor(ABC, Generic[T]):

    @abstractmethod
    def zero() -> T: pass

    @abstractmethod
    def one(currency: str) -> T: pass

    @abstractmethod
    def give(contract: Contract) -> T: pass

    @abstractmethod
    def and_(contract1: Contract, contract2: Contract) -> T: pass

    @abstractmethod
    def or_(contract1: Contract, contract2: Contract) -> T: pass

    @abstractmethod
    def truncate(horizon: datetime, contract: Contract) -> T: pass

    @abstractmethod
    def then(contract1: Contract, contract2: Contract) -> T: pass

    @abstractmethod
    def scale(observable: Observable, contract: Contract) -> T: pass

    @abstractmethod
    def get(contract: Contract) -> T: pass

    @abstractmethod
    def anytime(contract: Contract) -> T: pass

    def __call__(self, contract: Contract) -> T:
        if isinstance(contract, Zero):
            return self.zero()
        elif isinstance(contract, One):
            return self.one(**contract)

In [6]:
def scaleK(constant: float, contract: Contract):
    return Scale(Konst(constant), contract.__dict__)

In [37]:
def zcb(maturity: datetime, notional: float, currency: str):
    return scaleK(notional, Get(Truncate(maturity, One(currency))))

In [ ]:
class ContractPrinter(ContractVisitor[str]):
    

In [27]:
mybond1 = zcb(datetime(2030, 7, 14), 1000000, 'EUR')
mybond1

Scale(observable=Konst(constant=1000000), contract=Get(contract=Truncate(horizon=datetime.datetime(2030, 7, 14, 0, 0), contract=One(currency='EUR'))))

In [33]:
asdict(mybond1)

{'observable': {'constant': 1000000},
 'contract': {'contract': {'horizon': datetime.datetime(2030, 7, 14, 0, 0),
   'contract': {'currency': 'EUR'}}}}

In [35]:
astuple(mybond1)

((1000000,), ((datetime.datetime(2030, 7, 14, 0, 0), ('EUR',)),))

In [31]:
dict(**mybond1.__dict__)

{'observable': Konst(constant=1000000),
 'contract': Get(contract=Truncate(horizon=datetime.datetime(2030, 7, 14, 0, 0), contract=One(currency='EUR')))}